In [1]:
import numpy as np
from dependence import VineCopula
import itertools

In [26]:
def get_pair(dim, index):
    k = 0
    for i in range(1, dim):
        for j in range(i):
            if k == index:
                return [i+1, j+1]
            k+=1
            
def is_vine_structure(matrix):
    dim = matrix.shape[0]
    u_matrix = np.triu(matrix, k=1)
    diag = np.diag(matrix)
    assert matrix.max() == dim, "Maximum should be the dimension: %d != %d" % (matrix.max(), dim)
    assert matrix.min() == 0, "Minimum should be 0: %d != %d" % (matrix.min(), dim)
    assert matrix.shape[0] == matrix.shape[1], "Matrix should be squared"
    assert u_matrix.sum() == 0, "Matrix should be lower triangular"
    assert len(np.unique(diag)) == dim, "Element should be uniques on the diagonal: %d != %d" % (len(np.unique(diag)), dim)
    for i in range(dim):
        column_i = matrix[i:, i]
        assert len(np.unique(column_i)) == dim - i, "Element should be unique for each column: %d != %d" % ( len(np.unique(column_i)), dim - i)
        for node in diag[:i]:
            assert node not in column_i, "Previous main node should not exist after"
    return matrix

def check_diagonal(structure):
    if len(np.unique(np.diag(structure))) == n_forced_pairs:
        return True
    else:
        return False

def check_columns(structure):
    dim = structure.shape[0]
    diag = np.diag(structure)
    for i in range(dim):
        column_i = structure[i:, i]
        if len(np.unique(column_i)) != dim - i:
            return False
        for node in diag[:i]:
            if node in column_i:
                return False
            
def check_redundancy(structure):
    dim = structure.shape[0]
    diag = np.diag(structure)
    for i in range(dim-1):
        # Check if it does not appears later in the matrix
        if diag[i] != 0.:
            if diag[i] in structure[:, i+1:]:
                return False
    return True

def rotate_pairs(init_pairs, rotations):
    pairs = []
    for i, ci in enumerate(rotations):
        if ci == -1:
            pairs.append(list(reversed(init_pairs[i])))
        else:
            pairs.append(init_pairs[i])
    return pairs

def add_pair(structure, pair, index, lvl):
    assert structure.shape[0] == structure.shape[1], "Matrix should be squared"
    dim = structure.shape[0]
    if lvl == 0: # If it's the unconditiononal variables
        assert structure[index, index] == 0, "Diagonal should be empty"
        assert structure[dim-1, index] == 0, "Last row element should be empty"
        structure[index, index] = pair[0]
        structure[dim-1, index] = pair[1]
    else:
        assert structure[index, index] == pair[0], "First element should be the same as the first variable of the pair"
        structure[dim - 1 - lvl, index] = pair[1]
    return structure

def add_pairs(structure, pairs, lvl):
    """
    """
    dim = structure.shape[0]
    n_pairs = len(pairs)
    assert dim == structure.shape[1], "Matrix should be squared"
    assert n_pairs < dim - lvl, "Not enough place to fill the pairs"
    n_slots = dim - 1 - lvl
    possibilities = list(itertools.permutations(range(n_slots), r=n_pairs))
    for possibility in possibilities:
        try:
            # Add the pair in the possible order
            for i, pair in enumerate(pairs):
                structure = add_pair(structure, pair, possibility[i], lvl)
            break
        except AssertionError:
            pass

    dim = structure.shape[0]
    if (lvl == 0) and (n_pairs == dim-1):
        structure[dim-1, dim-1] = np.setdiff1d(range(dim+1), np.diag(structure))[0]
    return structure

def fill_structure(structure):
    """
    """
    prev_ind = []
    for i in range(dim):
        values_i = structure[i:, i]
        used_values_i = values_i[values_i != 0].tolist() + prev_ind
        remaining_i = range(1, dim + 1)
        for var in used_values_i:
            if (var in remaining_i):
                remaining_i.remove(var)
        values_i[values_i == 0] = remaining_i
        prev_ind.append(values_i[0])
    return structure

In [112]:
dim = 5
n_pairs = dim*(dim-1)/2
n_forced_pairs = 7
forced_pairs_ids = np.random.choice(range(n_pairs), n_forced_pairs, replace=False)
# forced_pairs_ids = [4, 5, 2, 3]
# n_forced_pairs = len(forced_pairs_ids)

In [152]:
assert len(np.unique(forced_pairs_ids)) == n_forced_pairs, "Unique values should be puted"
assert n_forced_pairs <= n_pairs, "Not OK!"
assert max(forced_pairs_ids) < n_pairs, "Not OK!"

n_conditionned = range(dim - 1, 0, -1)
forced_pairs = np.asarray([get_pair(dim, pair_id) for pair_id in forced_pairs_ids])
remaining_pairs_ids = range(0, n_pairs)
for pair_id in forced_pairs_ids:
    remaining_pairs_ids.remove(pair_id)
remaining_pairs = np.asarray([get_pair(dim, pair_id) for pair_id in remaining_pairs_ids])

print('Vine dimension: %d' % dim)
print('Conditioning information:')
k0 = 0
pairs_by_levels = []
for i in range(dim-1):
    k = n_conditionned[i]
    k1 = min(n_forced_pairs, k0+k)
    print("\t%d pairs with %d conditionned variables" % (k, i))
    print("Pairs: {0}".format(forced_pairs[k0:k0+k].tolist()))
    if forced_pairs[k0:k0+k].tolist():
        pairs_by_levels.append(forced_pairs[k0:k0+k].tolist())
    k0 = k1
print("Concerned pairs: {0}".format(forced_pairs.tolist()))
print("Remaining pairs: {0}".format(remaining_pairs.tolist()))

def check_node_loop(pairs):
    n_p = 3
    for perm_pairs in list(itertools.permutations(pairs, r=n_p)):
        if len(np.unique(perm_pairs)) <= n_p:
            return False
    return True

n_levels = len(pairs_by_levels)
lvl = 0
while lvl < n_levels:
    pairs_level = pairs_by_levels[lvl]
    if not check_node_loop(pairs_level):
        if lvl == n_levels - 1:
            pairs_by_levels.append([pairs_level.pop()])
            n_levels += 1
        else:
            pairs_by_levels[lvl+1].insert(0, pairs_level.pop())
            pairs_by_levels[lvl].append(pairs_by_levels[lvl+1].pop(1))
    lvl += 1
pairs_by_levels

Vine dimension: 5
Conditioning information:
	4 pairs with 0 conditionned variables
Pairs: [[5, 3], [5, 2], [3, 2], [5, 4]]
	3 pairs with 1 conditionned variables
Pairs: [[4, 2], [4, 3], [3, 1]]
	2 pairs with 2 conditionned variables
Pairs: []
	1 pairs with 3 conditionned variables
Pairs: []
Concerned pairs: [[5, 3], [5, 2], [3, 2], [5, 4], [4, 2], [4, 3], [3, 1]]
Remaining pairs: [[2, 1], [4, 1], [5, 1]]


[[[5, 3], [5, 2], [3, 2], [4, 2]], [[5, 4], [4, 3], [3, 1]]]

In [153]:
# For each levels
good_structures = []
for lvl, pairs_level in enumerate(pairs_by_levels):
    n_pairs_level = len(pairs_level) # Number of pairs in the level
    
    # The possible combinations
    combinations = list(itertools.product([1, -1], repeat=n_pairs_level))
    n_combinations = len(combinations)
    
    # Now lets get the possible pairs combinations for this level
    for k, comb_k in enumerate(combinations):
        # Rotate the pair to the actual combination
        pairs_k = rotate_pairs(pairs_level, comb_k)
        if lvl == 0:
            # Create the associated vine structure
            structure = np.zeros((dim, dim), dtype=int)
            structure = add_pairs(structure, pairs_k, lvl)
#             print structure
            if check_redundancy(structure):
                good_structures.append(structure)
        else:
            for structure in good_structures:
                try:
                    new_structure = add_pairs(structure, pairs_k, lvl)
                except:
                    print("Can't add the pairs {0} in the current structure...".format(pairs_k))
                    
                if check_redundancy(new_structure):
                    structure = new_structure
        
for structure in good_structures:
    print('good:\n{0}'.format(structure))

In [10]:
is_vine_structure(fill_structure(structure))

AssertionError: Element should be uniques on the diagonal: 4 != 5

In [113]:
all_good_pairs

[]

In [306]:
p = 1
good_pairs = all_good_pairs[p]
print len(all_good_pairs)
good_pairs

IndexError: list index out of range

In [302]:
structure = np.zeros((dim, dim), dtype=int)
for i, pair in enumerate(good_pairs):
    structure[i, i] = pair[0]
    structure[dim-1, i] = pair[1]
structure

array([[4, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 1, 0],
       [1, 3, 3, 0]])

In [303]:
prev_ind = []
for i in range(dim):
    values_i = structure[i:, i]
    used_values_i = values_i[values_i != 0].tolist() + prev_ind
    remaining_i = range(1, dim + 1)
    for var in used_values_i:
        if (var in remaining_i):
            remaining_i.remove(var)
    values_i[values_i == 0] = remaining_i
    prev_ind.append(values_i[0])
is_vine_structure(structure)

array([[4, 0, 0, 0],
       [2, 2, 0, 0],
       [3, 1, 1, 0],
       [1, 3, 3, 3]])